<a href="https://colab.research.google.com/github/Benjamin25-11/Tareas/blob/main/Tarea_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from jax import grad, random
from jax import jit
import jax.numpy as jnp

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalizamos los datos
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convertimos las imágenes a formato adecuado para el MLP
train_images = jnp.array(train_images.reshape(-1, 28 * 28))
test_images = jnp.array(test_images.reshape(-1, 28 * 28))
train_labels = jnp.array(train_labels)
test_labels = jnp.array(test_labels)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
class_names[train_labels[0]]

'Ankle boot'

In [4]:

# Función ReLU
@jit
def relu(z):
    return jnp.maximum(0, z)

# Función Softmax (para la salida de clasificación)
@jit
def softmax(z):
    exp_z = jnp.exp(z)
    return exp_z / jnp.sum(exp_z, axis=-1, keepdims=True)

# Definir MLP con una capa oculta
def mlp(params, x):
    w1, b1, w2, b2, w3, b3 = params
    # Capas ocultas
    z1 = jnp.dot(x, w1) + b1
    a1 = relu(z1)
    z2 = jnp.dot(a1, w2) + b2
    a2 = relu(z2)
    # Capa de salida
    z3 = jnp.dot(a2, w3) + b3
    return softmax(z3)

# Inicialización de pesos y sesgos
def inicializar_pesos(rng, input_size, hidden_sizes, output_size):
    w1 = random.normal(rng, (input_size, hidden_sizes[0])) * 0.01
    b1 = jnp.zeros((hidden_sizes[0],))
    w2 = random.normal(rng, (hidden_sizes[0], hidden_sizes[1])) * 0.01
    b2 = jnp.zeros((hidden_sizes[1],))
    w3 = random.normal(rng, (hidden_sizes[1], output_size)) * 0.01
    b3 = jnp.zeros((output_size,))
    return w1, b1, w2, b2, w3, b3

# Definir los tamaños de la red
input_size = 28 * 28  # Tamaño de la imagen aplanada
hidden_sizes = [300, 100]  # Tamaño de las capas ocultas
output_size = 10  # Número de clases

# Inicializar parámetros
rng = random.PRNGKey(0)
params = inicializar_pesos(rng, input_size, hidden_sizes, output_size)

In [5]:
# Función de pérdida (Entropía Cruzada)
@jit
def cross_entropy_loss(params, x, y):
    preds = mlp(params, x)
    return -jnp.mean(jnp.sum(y * jnp.log(preds), axis=-1))

# Función para calcular la precisión
@jit
def accuracy(params, x, y):
    predictions = jnp.argmax(mlp(params, x), axis=1)
    return jnp.mean(predictions == y)

# Función de retropropagación y actualización de pesos
@jit
def actualizar_pesos(params, x, y, learning_rate=0.01):
    # Calcular gradientes
    grads = grad(cross_entropy_loss)(params, x, y)
    # Actualizar los pesos
    params = [w - learning_rate * dw
              for w, dw in zip(params, grads)]
    return params

In [6]:
# Función de pérdida para calcular la pérdida en test
@jit
def test_loss(params, x, y):
    preds = mlp(params, x)
    return -jnp.mean(jnp.sum(y * jnp.log(preds), axis=-1))

# Configuración inicial
epochs = 30
learning_rate = 0.01
batch_size = 64
patience = 3  # Número de épocas sin mejora para detener
accuracies = []
losses = []

# Inicialización para early stopping
no_improvement_epochs = 0
best_loss = float('inf')

# Bucle de entrenamiento
for epoch in range(epochs):
    # Dividimos los datos en minibatches
    num_batches = len(train_images) // batch_size
    for i in range(num_batches):
        x_batch = train_images[i * batch_size:(i + 1) * batch_size]
        y_batch = jnp.eye(10)[train_labels[i * batch_size:(i + 1) * batch_size]]  # One-hot encoding

        # Actualizar los pesos usando el gradiente
        params = actualizar_pesos(params, x_batch, y_batch, learning_rate)

    # Evaluar la pérdida y precisión en el conjunto de prueba
    current_loss = test_loss(params, test_images, jnp.eye(10)[test_labels])
    test_acc = accuracy(params, test_images, test_labels)
    losses.append(current_loss)
    accuracies.append(test_acc)
    print(f'Época {epoch + 1}, Precisión en test: {test_acc:.3f}, Pérdida en test: {current_loss:.5f}')

    # Monitoreo para early stopping
    if jnp.abs(current_loss - best_loss) < 1e-2:  # Cambio menor a dos decimales
        no_improvement_epochs += 1
    else:
        no_improvement_epochs = 0
        best_loss = current_loss

    # Condición para detener el entrenamiento
    if no_improvement_epochs >= patience:
        print(f"Entrenamiento detenido temprano en la época {epoch + 1} debido a falta de mejora en la pérdida.")
        break


Época 1, Precisión en test: 0.180, Pérdida en test: 2.27458
Época 2, Precisión en test: 0.523, Pérdida en test: 1.23157
Época 3, Precisión en test: 0.650, Pérdida en test: 0.91221
Época 4, Precisión en test: 0.694, Pérdida en test: 0.83163
Época 5, Precisión en test: 0.725, Pérdida en test: 0.75784
Época 6, Precisión en test: 0.748, Pérdida en test: 0.68359
Época 7, Precisión en test: 0.765, Pérdida en test: 0.63980
Época 8, Precisión en test: 0.779, Pérdida en test: 0.61130
Época 9, Precisión en test: 0.790, Pérdida en test: 0.58962
Época 10, Precisión en test: 0.799, Pérdida en test: 0.57215
Época 11, Precisión en test: 0.804, Pérdida en test: 0.55717
Época 12, Precisión en test: 0.810, Pérdida en test: 0.54361
Época 13, Precisión en test: 0.814, Pérdida en test: 0.53091
Época 14, Precisión en test: 0.818, Pérdida en test: 0.51917
Época 15, Precisión en test: 0.820, Pérdida en test: 0.50874
Época 16, Precisión en test: 0.823, Pérdida en test: 0.49914
Época 17, Precisión en test: 0.82